# Школа глубокого обучения

<a href="https://mipt.ru/science/labs/laboratoriya-neyronnykh-sistem-i-glubokogo-obucheniya/"><img align="right" src="https://avatars1.githubusercontent.com/u/29918795?v=4&s=200" alt="DeepHackLab" style="position:relative;top:-40px;right:10px;height:100px;" /></a>



### Физтех-Школа Прикладной математики и информатики МФТИ 
### Лаборатория нейронных сетей и глубокого обучения (DeepHackLab)

*Илья Захаркин (ФИВТ МФТИ)*

## Работа с текстовыми данными

Первая половина этого семестра в основном про NLP (Natural Language Processing), а знаитч перед тем, как нырять в LSTM и прочее, неплохо бы научиться работать с текстом и его представлениями.

Ноутбук соотвтествует лекции, то есть тут приведён код для большинства того, что было на лекции, чтобы вы представляли, как всё это выглядит в коде. Поэтому по порядку:

### Категориальные признаки

* LabelEncoder:

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
fruits = pd.DataFrame(data={'Фрукт': ['яблоко', 'груша', 'яблоко', 'банан', 'киви']})

In [4]:
fruits

,Фрукт
0,яблоко
1,груша
2,яблоко
3,банан
4,киви


In [5]:
encoder = LabelEncoder()
result = encoder.fit_transform(fruits.values.reshape(-1,))

In [6]:
fruits['LabelEncoder'] = result
fruits

,Фрукт,LabelEncoder
0,яблоко,3
1,груша,1
2,яблоко,3
3,банан,0
4,киви,2


### Текстовые данные

In [1]:
import numpy as np
import pandas as pd

import nltk

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [2]:
!pip install nltk

### Скачиваем данные

Привет!  

В этом ноутбуке мы проанализируем датасет отзывов с *IMDB*, будем пытаться оценить их позитивность (1) и негативность (0).

In [3]:
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Илья\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [4]:
from nltk.corpus import movie_reviews

* Информация о датасете::

In [5]:
print(movie_reviews.readme())

Sentiment Polarity Dataset Version 2.0
Bo Pang and Lillian Lee

http://www.cs.cornell.edu/people/pabo/movie-review-data/

Distributed with NLTK with permission from the authors.


Introduction

This README v2.0 (June, 2004) for the v2.0 polarity dataset comes from
the URL http://www.cs.cornell.edu/people/pabo/movie-review-data .


What's New -- June, 2004

This dataset represents an enhancement of the review corpus v1.0
described in README v1.1: it contains more reviews, and labels were
created with an improved rating-extraction system.


Citation Info 

This data was first used in Bo Pang and Lillian Lee,
``A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization 
Based on Minimum Cuts'',  Proceedings of the ACL, 2004.

@InProceedings{Pang+Lee:04a,
  author =       {Bo Pang and Lillian Lee},
  title =        {A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts},
  booktitle =    "Proceedings of the ACL",
  year =      

* ЧТобы удобно было разделять отзывы:

In [6]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [7]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [8]:
negfeats[:5]

[['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...],
 ['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...],
 ['it', 'is', 'movies', 'like', 'these', 'that', 'make', ...],
 ['"', 'quest', 'for', 'camelot', '"', 'is', 'warner', ...],
 ['synopsis', ':', 'a', 'mentally', 'unstable', 'man', ...]]

In [9]:
posfeats[-5:]

[['wow', '!', 'what', 'a', 'movie', '.', 'it', "'", 's', ...],
 ['richard', 'gere', 'can', 'be', 'a', 'commanding', ...],
 ['glory', '--', 'starring', 'matthew', 'broderick', ...],
 ['steven', 'spielberg', "'", 's', 'second', 'epic', ...],
 ['truman', '(', '"', 'true', '-', 'man', '"', ')', ...]]

* Саздаём **DataFrame** и вектор меток **y**:

In [29]:
sentiments = pd.DataFrame({'sentiment': negfeats + posfeats})  # all sentiments
labels = np.array([0] * len(negfeats) + [1] * len(posfeats))  # 0 - bad, 1 - good

In [30]:
sentiments.head()

,sentiment
0,"(plot, :, two, teen, couples, go, to, a, churc..."
1,"(the, happy, bastard, ', s, quick, movie, revi..."
2,"(it, is, movies, like, these, that, make, a, j..."
3,"("", quest, for, camelot, "", is, warner, bros, ..."
4,"(synopsis, :, a, mentally, unstable, man, unde..."


In [31]:
sentiments.tail()

,sentiment
1995,"(wow, !, what, a, movie, ., it, ', s, everythi..."
1996,"(richard, gere, can, be, a, commanding, actor,..."
1997,"(glory, --, starring, matthew, broderick, ,, d..."
1998,"(steven, spielberg, ', s, second, epic, film, ..."
1999,"(truman, (, "", true, -, man, "", ), burbank, is..."


In [32]:
sentiments.shape

(2000, 1)

In [33]:
len(posfeats) / (len(negfeats) + len(posfeats))

0.5

### Преобразование данных в ***bag-of-words*** (мешок слов):

In [34]:
cnt_vect = CountVectorizer()

In [35]:
data = [' '.join(sent) for sent in sentiments['sentiment']]

In [36]:
X = cnt_vect.fit_transform(data)

In [37]:
X

<2000x39659 sparse matrix of type '<class 'numpy.int64'>'
	with 666842 stored elements in Compressed Sparse Row format>

In [38]:
y = labels

### Пробуем логистическую регрессию ***LogisticRegression*** как модель для классификации:

In [20]:
clf = LogisticRegression()

In [21]:
vect_logreg = Pipeline([('vectorizer', cnt_vect), ('logreg', clf)])

* Измерим качество **accuracy score**:

In [22]:
acc_scores = cross_val_score(vect_logreg, data, y, scoring='accuracy')

In [23]:
np.mean(acc_scores)

0.83602165039290777

* Измерим качество с помощью **ROC AUC**:

In [24]:
roc_aucs = cross_val_score(vect_logreg, data, y, scoring='roc_auc')

In [25]:
np.mean(roc_aucs)

0.91077649378337744

* Обучаем модель на *всех данных*:

In [39]:
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Top-5 наиболее важных слов для классификации:

In [27]:
idx_arr = np.argsort(np.abs(clf.coef_))[0][::-1][:5]
idx_arr

array([ 2954, 37056, 39195, 14159, 38417], dtype=int64)

In [28]:
np.array(cnt_vect.get_feature_names())[idx_arr]

array(['bad', 'unfortunately', 'worst', 'fun', 'waste'],
      dtype='<U58')

---

### Часть 2. Экспериментируем с моделями, stop-words и n-grams

* Давайте провреим, какое качтсов даст бейзлайн **без stop-words и n-grams**:

In [40]:
count_logreg_pipe = Pipeline([('vectorizer', CountVectorizer()), ('logreg', LogisticRegression())])
tfidf_logreg_pipe = Pipeline([('vectorizer', TfidfVectorizer()), ('logreg', LogisticRegression())])

In [41]:
cv_acc_scores_count = cross_val_score(count_logreg_pipe, data, y, cv=5)
cv_acc_scores_tfidf = cross_val_score(tfidf_logreg_pipe, data, y, cv=5)

In [42]:
print('CountVectorizer + LogisticRegression CV accuracy: {:.5f} ({:.5f})'.format(cv_acc_scores_count.mean(), 
                                                                                 cv_acc_scores_count.std()))

CountVectorizer + LogisticRegression CV accuracy: 0.84150 (0.01678)


In [43]:
print('TfIdfVectorizer + LogisticRegression CV accuracy: {:.5f} ({:.5f})'.format(cv_acc_scores_tfidf.mean(), 
                                                                                 cv_acc_scores_tfidf.std()))

TfIdfVectorizer + LogisticRegression CV accuracy: 0.82100 (0.00406)


* Теперь изменим *min_df* у *CountVectorizer* и увидим, как изменится модель:

In [44]:
param_grid = [10, 50]

for min_df in param_grid:
    count_logreg_pipe = Pipeline([('vectorizer', CountVectorizer(min_df=min_df)), ('logreg', LogisticRegression())])
    cv_acc_scores_count = cross_val_score(count_logreg_pipe, data, y, cv=5)
    print('CountVectorizer + LogisticRegression CV accuracy with min_df={:d}: {:.3f} ({:.3f})\n'.format(min_df,
                                                                                                    cv_acc_scores_count.mean(), 
                                                                                                    cv_acc_scores_count.std()))

CountVectorizer + LogisticRegression CV accuracy with min_df=10: 0.839 (0.012)

CountVectorizer + LogisticRegression CV accuracy with min_df=50: 0.813 (0.013)



* Попробуем разные модели:

In [117]:
classifiers = [LogisticRegression(), LinearSVC(), SGDClassifier()]
pipelines = [Pipeline([('vectorizer', CountVectorizer()), ('classifier', clf)]) for clf in classifiers]

In [118]:
for pipeline in pipelines:
    cv_acc_scores = cross_val_score(pipeline, data, y, cv=5)
    print('CV accuracy: {:f} ({:f})\n'.format(cv_acc_scores.mean(), cv_acc_scores.std()))

CV accuracy: 0.841000 (0.016778)

CV accuracy: 0.832500 (0.016279)

CV accuracy: 0.761000 (0.044598)



* Сравним **stop-words** из **sklearn** и из **nltk**:

In [91]:
stop_words_nltk = nltk.corpus.stopwords.words('english')

In [96]:
stop_words = ['english', stop_words_nltk]

pipelines = [Pipeline([('vectorizer', CountVectorizer(stop_words=words)), ('logreg', LogisticRegression())]) 
             for words in stop_words]

names = ['sklearn', 'nltk']
for i, pipeline in enumerate(pipelines):
    cv_acc_scores = cross_val_score(pipeline, data, y, cv=5)
    print('CountVectorizer + LogisticRegression + stop-words: {} CV accuracy: {:.3f} ({:.3f})'.format(names[i],
                                                                                                      cv_acc_scores.mean(), 
                                                                                                      cv_acc_scores.std()))

CountVectorizer + LogisticRegression + stop-words: sklearn CV accuracy: 0.839 (0.010)
CountVectorizer + LogisticRegression + stop-words: nltk CV accuracy: 0.841 (0.010)


* И, наконец, попробуем **1-2-word-grams** и **3-5-word-grams**:

In [107]:
pipeline = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 2))),
                     ('logreg', LogisticRegression())])

In [108]:
cv_acc_scores = cross_val_score(pipeline, data, y, cv=5)
print('CountVectorizer + LogisticRegression + 1-2-word-grams CV accuracy: {:.3f} ({:.3f})'.format(cv_acc_scores.mean(), 
                                                                                                  cv_acc_scores.std()))

CountVectorizer + LogisticRegression + 1-2-word-grams CV accuracy: 0.853 (0.017)


In [110]:
pipeline = Pipeline([('vectorizer', CountVectorizer(analyzer='char_wb', ngram_range=(3, 5))), 
                     ('logreg', LogisticRegression())])

In [111]:
cv_acc_scores = cross_val_score(pipeline, data, y, cv=5)
print('CountVectorizer + LogisticRegression + 3-5-letter-grams CV accuracy: {:.3f} ({:.3f})'.format(cv_acc_scores.mean(), 
                                                                                                    cv_acc_scores.std()))

CountVectorizer + LogisticRegression + 3-5-letter-grams CV accuracy: 0.820 (0.011)
